In [1]:
# Importamos las librerías necesarias
import datetime      # Para trabajar con las fechas (cada bloque tiene su propio timestamp)
import hashlib       # Para crear los hashes de los bloques, mediante el SHA256
import json          # Para crear ficheros .json, muchas funciones antes de "hashear" estarán en este formato
from flask import Flask   # Constructor para crear objetos tipo Flask
from flask import jsonify  # Para convertir a .json y que Postman utilizará para interactuar con la cadena de bloques

## Parte 1 - Crear la cadena de bloques

Primero se define la clase `Blockchain`, que será el **bloque génesis**, es decir, el bloque con el que se inicializa la cadena de bloques y del que se originará toda la cadena en sí misma.

NOTA: no confundir la función `create_block` con la de minar bloques. Para los bloques sucesivos de la cadena, primero hay que minarlos, es decir, resolver el puzzle criptográfico averiguando el NONCE con el que se crea un nuevo bloque y se obtiene la recompensa corresponde. Una vez minado un bloque, este se CREA mediante `create_block` y se añade a la cadena. 

A esta función se le pasan dos argumentos: el **Proof of Work** y el **hash previo** del bloque anterior.

Respecto al problema criptográfico a resolver, no se van a poner muchos ceros a la izquierda, ya que mientras más ceros haya, más difícil será de resolver el problema. En el ejemplo de código, se pondrán **5 ceros** para que no tenga excesiva complejidad el problema a resolver. 

En la línea de código `hashlib.sha256()`, hay que procurar utilizar una operación **no simétrica**, ya que si es simétrica el problema será muy fácil de resolver (por ejemplo, no podemos sumar el hash previo y el hash actual, porque daría el mismo resultado a la inversa, y entraríamos en un bucle infinito). Tampoco debe ser una operación muy compleja, ya que si es muy enrevesada, se consumirá mucho tiempo en resolver el problema.

A modo de prueba, se ha puesto la operación de `new_proof**2 - previous_proof**2`, que no es simétrica, ya que no se obtiene el mismo resultado con $5^2 - 2^2$ que con $2^2 - 5^2$. El resultado será **un número**, hay que pasar el resultado a `string`. Pero no vale con convertirlo, también hay que codificarlo mediante `.encode()`, para que el resultado pueda ser interpretado por el algoritmo SHA256. 

Dado que el algoritmo nos devolverá un valor codificado, a la salida de la ecuación criptográfica hay que añadir una función `hexdigest()`, que convertirá el valor en **hexadecimal**.

En la función `def hash()`, el *bloque* es un diccionario, de modo que habrá que convertirlo a string codificado para poder suministrárselo al algoritmo SHA256. Pero no se va a codificar directamente en string como en la función `proof_of_work`, sino que se va a volcar el diccionario en un .JSON, porque el propio .JSON ya tiene la posibilidad de ser codificado directamente en un formato binario que se la pueda pasar como input al algoritmo SHA256. Con la función `json.dumps()` podemos hacer la conversión, en este caso, de diccionario a string, y luego mediante `.encode()` se codificar para que tenga el formato adecuado para el SHA256.

Luego se crea una función para validar si la cadena de bloques es válida o no es válida, y para ello se comprueban dos cosas:
  * Para cada bloque, hay que comprobar que el campo `previous_hash` con el hash del bloque inmediatamente anterior.
  * Para cada uno de los bloques, hay que verificar la `proof_of_work`, es decir, si supera la ecuación algorítmica.

In [2]:
# PARTE 1: CREACIÓN DE LA CADENA DE BLOQUES

class Blockchain: 
    
    # Constructor de la cadena de bloques
    def __init__(self):
        self.chain = []      # Es una lista donde se almacenarán todos los bloques a medida que se vayan creando
        self.create_block(proof = 1, previous_hash = '0')   # Se asigna el valor 1 ya que será el primer valor de la cadena
                                                      # Como el bloque génesis no tiene hash previo, el valor será 0 (en char)
    # Crear un nuevo bloque de la cadena  
    def create_block(self, proof, previous_hash):
        # Creamos un bloque que será un diccionario con claves y valores, para convertirlo fácilmente a .json
        block = {'index' : len(self.chain)+1,   # Será la a longitud de la cadena añadiendo el nuevo bloque (+1) 
                 'timestamp' : str(datetime.datetime.now()),  # Se convierte a string para luego no tener problemas de formato
                 'proof' : proof,           # El Proof of Work ya estará creado cuando se le pase a la función
                 'previous_hash' : previous_hash}   # Lo mismo con el hash previo: ya estará creado
        self.chain.append(block)
        return block   # Así cuando se utilice Postman se podrán mostrar los datos del bloque creado
    
    # Obtener el último bloque de la cadena
    def get_previous_block(self):
        return self.chain[-1]      # Se obtiene el último bloque de la cadena
    
    # Prueba de Trabajo (PROOF OF WORK)
    def proof_of_work(self, previous_proof):   # Se necesita la solución del problema previo porque se utilizará para crear el problema siguiente
        new_proof = 1
        check_proof = False   # Cuando se haya resuelto el problema criptográfico, tendrá valor TRUE
        while check_proof is False:
            hash_operation = hashlib.sha256(str(new_proof**2 - previous_proof**2).encode())
            if hash_operation[:5] == '00000':
                check_proof = True
            else:
                new_proof += 1  # Sumamos unidad a unidad hasta que se cumpla la condición (que el hash tenga 5 ceros)
    
    # FUNCIÓN DE HASH
    def HASH(self, block):  # Dos argumentos: "self" para acceder a todo lo relativo a la clase y un bloque
        encoded_block = json.dumps(block, sort_keys=True).encode()  # con sort_keys nos aseguramos que las claves del diccionario vengan en el mismo orden
        return hashlib.sha256(encoded_block).hexdigest()  # Aquí se crear el hash con algoritmo SHA256, y hexdigest para que se vea en hexadecimal
    
    # COMPROBAR SI LA CADENA DE BLOQUES ES VÁLIDA
    def is_chain_valid(self, chain):
        previous_block = chain[0]  # Le asignamos el bloque que sabemos que siempre va a existir: el bloque génesis
        block_index = 1  # El bloque génesis no hace falta comprobarlo, hay que comprobar el primer bloque después del inicial
        while block_index < len(chain):  # Mientras el índice del bloque sea menor estricto (<) que la longitud de la cadena
            block = chain[block_index]  # La cadena en la posición "block_index" nos da exactamente el bloque que vamos a procesar
            if block['previous_hash'] != self.HASH(previous_block):
                return False                            # No supera la primera condición
            
            previous_proof = previous_block['proof']   # El valor de la prueba previa
            proof = block['proof']                     # El valor de la prueba actual
            hash_operation = hashlib.sha256(str(proof**2 - previous_proof**2).encode())  
            # Es la misma operación que en la función proof_of_work (solo hemos cambiado el nombre de la variable "new_proof" por "proof")
            
            if hash_operation[:5] != '00000': # Si las 5 primeras posiciones son distintas a 00000, la cadena será inválida
                return False    # No supera la segunda condición (si el eslabón de la cadena es válido)
            previous_block = block   # Si el bucle se supera con éxito, el bloque actual se convierte en el anterior
            block_index += 1         # Como hemos avanzando un eslabón en la cadena, se añade una unidad al índice del bloque


## Parte 2 - Minado de una cadena de bloques

Esta segunda parte se va a dividir en dos subapartados:

  * se va a crear **una aplicación web (web app)** basada en `Flask`, para poder interactuar con la blockchain, realizar peticiones a través de la interfaz de Postman, etc.
  * se va a crear un objeto de la clase `blockchain`, es decir, **se va a crear una cadena de bloques**, aprovechando que ya tenemos la clase para hacerlo (Parte 1). 
  
Esta cadena de bloques será un objeto que se va a instanciar de la clase `blockchain`, y será el que luego se va a utilizar para poner en producción, para sincronizar a través de `Flask` en una aplicación web en varios nodos.

A partir de aquí, con `Flask` se podrán hacer dos peticiones: la primera será minar un bloque de la cadena y la segunda será obtener toda la cadena de bloques. (Documentación Flask: https://flask.palletsprojects.com/en/1.1.x/quickstart/ )

La URL sobre la que funcionará (*Local Host*) en nuestra máquina en local será `http://127.0.0.1:5000`, donde $5000$ será el **puerto** de la URL. Al final de esta URL, se pueden añadir uno de los siguientes "puntos base" (o de ancla) según la petición que queramos hacer al servidor:

 * `/mine_block` será el "punto de base" (o de ancla) para la URL que se utilizará para Postman.
 * `/`

En `methods`, se podrán usar dos argumentos: `GET` que sirve para **obtener** información, y `POST` que sirve para **crear** nueva información. Para poder interactuar con Postman, de momento solo se utilizará **GET**.
 
Luego, para minar un bloque con la función `mine_block()` se necesitará el bloque previo, la prueba del bloque anterior, la prueba de trabajo del bloque actual, el hash previo y finalmente se creará el nuevo bloque `block`. Una vez creado un nuevo bloque, se mostrará por pantalla la **respuesta del servidor** (`response`), la cual contendrá todos los campos del diccionario con la información del bloque y añadirá una respuesta para notificar al usuario la creación de un nuevo bloque. ¿Por qué un diccionario? Porque cuando se hacen peticiones a través de *Web Service*, es el formato adecuado para poder darle a cada variable un nombre. 

Esta variable `response` contendrá las 4 variables del bloque + 1 variable "respuesta" para informar sobre el minado del nuevo bloque, la cual es un diccionario que se convertirá a un objeto .json (con la función `jsonify`) para que **Postman** pueda interpretar y mostrar al usuario cuando haya finalizado el proceso de minado del bloque.

De acuerdo a los **códigos de estado de respuesta HTTP**, nos interesa mostrar el código **200**, que se emplea para indicar que la solicitud ha tenido éxito. https://developer.mozilla.org/es/docs/Web/HTTP/Status

In [4]:
# PARTE 2 - MINADO DE UN BlOQUE DE LA CADENA

#  Crear una aplicación web (web app)
app = Flask(__name__)

#  Crear una blockchain
blockchain = Blockchain()
app.config['JSONIFY_PRETTYPRINT_REGULAR'] = False

#### ------------------------------------------------------------------------

# PRIMERA PETICIÓN: minado de un nuevo bloque
@app.route('/mine_block', methods = ['GET'])

def mine_block():
    previous_block = blockchain.get_previous_block()
    previous_proof = previous_block['proof']
    proof = blockchain.proof_of_work(previous_proof)
    previous_hash = blockchain.hash(previous_block)
    block = blockchain.create_block(proof, previous_hash)
    # Una vez creado el bloque, añadimos una respuesta:
    response = {'message' : 'Se ha minado un nuevo bloque',
                'index' : block['index'],
                'timestamp' : block['timestamp'],
                'proof' : block['proof'],
                'previous_hash' : block['previous_hash']}
    return jsonify(response), 200

#### ------------------------------------------------------------------------

# SEGUNDA PETICIÓN: obtener la cadena de bloques entera
@app.route('/get_chain', methods = ['GET'])

def get_chain():  
    response = {'chain' : blockchain.chain,
                'longitud cadena' : len(blockchain.chain)}
    
    return jsonify(response), 200

NOTA: si se obtiene un error `500 Internal Server Error`, hay que actualizar la librería Flask, reiniciar Spyder/Jupyter/etc y ejecutar la siguiente línea de código (se puede incluir con el resto del código):

`app.config['JSONIFY_PRETTYPRINT_REGULAR'] = False`

---

## Ejecutar la app

In [ ]:
app.run(host = '0.0.0.0', port = 5000)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.42.233:5000 (Press CTRL+C to quit)
127.0.0.1 - - [06/Jul/2022 01:10:27] "GET /get_chain HTTP/1.1" 200 -
[2022-07-06 01:10:39,385] ERROR in app: Exception on /mine_block [GET]
Traceback (most recent call last):
  File "C:\Users\romej\anaconda3\lib\site-packages\flask\app.py", line 2077, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\romej\anaconda3\lib\site-packages\flask\app.py", line 1525, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\romej\anaconda3\lib\site-packages\flask\app.py", line 1523, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\romej\anaconda3\lib\site-packages\flask\app.py", line 1509, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "<ipython-input-4-0eb33199dfe2>", line 18, in mine_block
    proof = blockchain.proof_of_work